##   Task 2: Natural Language Processing (NLP) - Text Classification


- Description: Classify text data into categories (e.g.,
spam vs. non-spam, sentiment analysis).

In [58]:
import transformers
import torch
import pandas as pd
import numpy as np
# import huggingface_hub['hf_xet']

In [59]:
from transformers import pipeline

# Load a sentiment-analysis pipeline
classifier = pipeline("sentiment-analysis")

# Run the model
result = classifier("The film was no good")
print(result[0]['label'])


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


NEGATIVE


- Preprocess text data (tokenization, removing
stopwords, stemming/lemmatization).

    - Tokenization

            Tokenization is the process of spliting the whole text into small pieces like tokens, for example I Love NLP to 'I', 'Love', 'NLP'.
    - Removing Stop words.

            Stopwords are common words that usually don't add significant meaning to a sentence, such as "is", "the", "and", etc. Removing them can help focus on the important words.

    - stemming

            Stemming reduces words to their root form by removing suffixes. It's a crude way of getting to the base of a word.


    - Lemmatization

            Lemmatization also reduces words to their base form, but it does so using vocabulary and morphological analysis. It ensures that the root word is a valid word.

In [60]:
## Tokenization
# extracting the text from our pdf document
import PyPDF2
with open(r"D:\codeveda_internship\Level 3 (Advanced)\National AI Policy Consultation Draft V1.pdf", "rb") as file:
    reader = PyPDF2.PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
text

' \n \ni\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n                        Draft\nNational\n \nArtificial Intelligence Policy\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nGovernment of Pakistan\nMinistry of Information Technology & Telecommunication\nhttps://moitt.gov.pk\n \n \n \n \n \nii \n Acknowledgments  \nThe Government of Pakistan, Ministry of IT & Telecom , pays its gratitude to all the officials  and consultants, \nparticularly  RSM Pakistan and GlowBug Technologies (Pvt.) Ltd. , facilitators, developers, and stakeholders who \nrigorously and relentlessly participated in the revi ew, drafting, harmonizing, and ratification of the National \nArtificial Intelligence Policy – 2022, helping the Ministry with an all -inclusive user -centric, evidence -based, \nforward -looking , and agile policy framework for enabling Pakistan towards a digital economy and society.  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \

In [61]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokens = tokenizer.tokenize(text)
print(tokens)



['i', 'draft', 'national', 'artificial', 'intelligence', 'policy', 'government', 'of', 'pakistan', 'ministry', 'of', 'information', 'technology', '&', 'telecommunication', 'https', ':', '/', '/', 'moi', '##tt', '.', 'gov', '.', 'p', '##k', 'ii', 'ac', '##k', '##now', '##led', '##gm', '##ents', 'the', 'government', 'of', 'pakistan', ',', 'ministry', 'of', 'it', '&', 'telecom', ',', 'pays', 'its', 'gratitude', 'to', 'all', 'the', 'officials', 'and', 'consultants', ',', 'particularly', 'rs', '##m', 'pakistan', 'and', 'glow', '##bu', '##g', 'technologies', '(', 'pv', '##t', '.', ')', 'ltd', '.', ',', 'fa', '##ci', '##lita', '##tors', ',', 'developers', ',', 'and', 'stakeholders', 'who', 'rigorous', '##ly', 'and', 'relentless', '##ly', 'participated', 'in', 'the', 'rev', '##i', 'e', '##w', ',', 'drafting', ',', 'harmon', '##izing', ',', 'and', 'ratification', 'of', 'the', 'national', 'artificial', 'intelligence', 'policy', '–', '202', '##2', ',', 'helping', 'the', 'ministry', 'with', 'an', 

In [62]:
len(tokens)

25551

In [63]:
# Stopword Removal
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords if not already done
nltk.download('stopwords')

# Get the list of stopwords in English
stop_words = set(stopwords.words('english'))
# Remove stopwords from the tokens
filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
print("Filtered Tokens:", filtered_tokens)


Filtered Tokens: ['draft', 'national', 'artificial', 'intelligence', 'policy', 'government', 'pakistan', 'ministry', 'information', 'technology', '&', 'telecommunication', 'https', ':', '/', '/', 'moi', '##tt', '.', 'gov', '.', 'p', '##k', 'ii', 'ac', '##k', '##now', '##led', '##gm', '##ents', 'government', 'pakistan', ',', 'ministry', '&', 'telecom', ',', 'pays', 'gratitude', 'officials', 'consultants', ',', 'particularly', 'rs', '##m', 'pakistan', 'glow', '##bu', '##g', 'technologies', '(', 'pv', '##t', '.', ')', 'ltd', '.', ',', 'fa', '##ci', '##lita', '##tors', ',', 'developers', ',', 'stakeholders', 'rigorous', '##ly', 'relentless', '##ly', 'participated', 'rev', '##i', 'e', '##w', ',', 'drafting', ',', 'harmon', '##izing', ',', 'ratification', 'national', 'artificial', 'intelligence', 'policy', '–', '202', '##2', ',', 'helping', 'ministry', '-', 'inclusive', 'user', '-', 'cent', '##ric', ',', 'evidence', '-', 'based', ',', 'forward', '-', 'looking', ',', 'agile', 'policy', 'frame

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
len(filtered_tokens)

19740

Our tokens after stopwords reduced to 19740 from 25551.

In [65]:
# Stemming
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
stemmed_tokens = [stemmer.stem(stemed_token) for stemed_token in filtered_tokens]
print("Stemmed Tokens:", stemmed_tokens)

Stemmed Tokens: ['draft', 'nation', 'artifici', 'intellig', 'polici', 'govern', 'pakistan', 'ministri', 'inform', 'technolog', '&', 'telecommun', 'http', ':', '/', '/', 'moi', '##tt', '.', 'gov', '.', 'p', '##k', 'ii', 'ac', '##k', '##now', '##led', '##gm', '##ent', 'govern', 'pakistan', ',', 'ministri', '&', 'telecom', ',', 'pay', 'gratitud', 'offici', 'consult', ',', 'particularli', 'rs', '##m', 'pakistan', 'glow', '##bu', '##g', 'technolog', '(', 'pv', '##t', '.', ')', 'ltd', '.', ',', 'fa', '##ci', '##lita', '##tor', ',', 'develop', ',', 'stakehold', 'rigor', '##li', 'relentless', '##li', 'particip', 'rev', '##i', 'e', '##w', ',', 'draft', ',', 'harmon', '##ize', ',', 'ratif', 'nation', 'artifici', 'intellig', 'polici', '–', '202', '##2', ',', 'help', 'ministri', '-', 'inclus', 'user', '-', 'cent', '##ric', ',', 'evid', '-', 'base', ',', 'forward', '-', 'look', ',', 'agil', 'polici', 'framework', 'enabl', 'pakistan', 'toward', 'digit', 'economi', 'societi', '.', 'iii', 'tabl', 'con

In [66]:
# lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmetizer = WordNetLemmatizer()
lemmatized_tokens = [lemmetizer.lemmatize(token) for token in filtered_tokens]
print("Lemmatized Tokens:", lemmatized_tokens)

Lemmatized Tokens: ['draft', 'national', 'artificial', 'intelligence', 'policy', 'government', 'pakistan', 'ministry', 'information', 'technology', '&', 'telecommunication', 'http', ':', '/', '/', 'moi', '##tt', '.', 'gov', '.', 'p', '##k', 'ii', 'ac', '##k', '##now', '##led', '##gm', '##ents', 'government', 'pakistan', ',', 'ministry', '&', 'telecom', ',', 'pay', 'gratitude', 'official', 'consultant', ',', 'particularly', 'r', '##m', 'pakistan', 'glow', '##bu', '##g', 'technology', '(', 'pv', '##t', '.', ')', 'ltd', '.', ',', 'fa', '##ci', '##lita', '##tors', ',', 'developer', ',', 'stakeholder', 'rigorous', '##ly', 'relentless', '##ly', 'participated', 'rev', '##i', 'e', '##w', ',', 'drafting', ',', 'harmon', '##izing', ',', 'ratification', 'national', 'artificial', 'intelligence', 'policy', '–', '202', '##2', ',', 'helping', 'ministry', '-', 'inclusive', 'user', '-', 'cent', '##ric', ',', 'evidence', '-', 'based', ',', 'forward', '-', 'looking', ',', 'agile', 'policy', 'framework', 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


- Convert text into numerical representation using TF-IDF
or Word2Vec.


In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidfvector_metrics = vectorizer.fit_transform([text])
print(tfidfvector_metrics.shape)

(1, 2846)


- Train a classification model (e.g., Naive Bayes, Logistic
Regression) on the processed text.



In [68]:
dataset = pd.read_csv("D:\codeveda_internship\Level 3 (Advanced)\IMDB Dataset.csv")
dataset.head()

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Admin\AppData\Local\Temp\ipykernel_16528\612012017.py:1: SyntaxWarning: invalid escape sequence '\c'
  dataset = pd.read_csv("D:\codeveda_internship\Level 3 (Advanced)\IMDB Dataset.csv")


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


We have use IMDB 50K dataset for text classification model training and predicting.

In [69]:
# we use TfidfVectorizer class for preprocessing the text data it firstly convertes the text into tokens and then convert it into embeddings.

vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(dataset['review'])
y = dataset['sentiment']

In [70]:
print(x)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6826529 stored elements and shape (50000, 101895)>
  Coords	Values
  (0, 64131)	0.016499383457650688
  (0, 63757)	0.07772820644793366
  (0, 90160)	0.1702648103348755
  (0, 64776)	0.04889158440621628
  (0, 75511)	0.059914725990916795
  (0, 40745)	0.020245420527016073
  (0, 57558)	0.051657767603700104
  (0, 90137)	0.051109066555307094
  (0, 2970)	0.02617714707391057
  (0, 98226)	0.06081671753720301
  (0, 48473)	0.039406217513103525
  (0, 65469)	0.4204842552200601
  (0, 30113)	0.0902220815013764
  (0, 101096)	0.050687330387343854
  (0, 53223)	0.10579810521305968
  (0, 8758)	0.033179033787529476
  (0, 42850)	0.06554850976141105
  (0, 90347)	0.019384864480591957
  (0, 5707)	0.03361745348626366
  (0, 75915)	0.0676568066832172
  (0, 6166)	0.06107841639019563
  (0, 90455)	0.03477363283267793
  (0, 46765)	0.10551468919685113
  (0, 30975)	0.045538626745490994
  (0, 98847)	0.040596904597151935
  :	:
  (49999, 52022)	0.08563776754631129

In [71]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [72]:
# Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)

MultinomialNB()

In [74]:
# Naive Bayes LogisticRegression
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model.fit(x_train, y_train)

LogisticRegression()

- Evaluate the model using precision, recall, and F1-score.


In [75]:
# precision recall and f1 score of naive bayes classifier
from sklearn.metrics import accuracy_score, classification_report
nb_preds = nb_model.predict(x_test)
print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_preds))
print(classification_report(y_test, nb_preds))

Naive Bayes Accuracy: 0.8638
              precision    recall  f1-score   support

    negative       0.84      0.89      0.87      4961
    positive       0.88      0.84      0.86      5039

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



In [76]:
# precision recall and f1 score of logisticregression classifier
from sklearn.metrics import accuracy_score, classification_report
lr_preds = lr_model.predict(x_test)
print("Naive Bayes Accuracy:", accuracy_score(y_test, lr_preds))
print(classification_report(y_test, lr_preds))

Naive Bayes Accuracy: 0.8997
              precision    recall  f1-score   support

    negative       0.91      0.89      0.90      4961
    positive       0.89      0.91      0.90      5039

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



- Tools: Python, nltk, scikit-learn, pandas.